<a href="https://colab.research.google.com/github/nageswar307/LLMFromScratch/blob/main/LLMFromScratchPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00


1. get raw input text for training
2. tokenization
3. prepare input and target sequence
4. creating token embeddings


##1. get raw input  text for training

In [3]:
with open("/content/drive/MyDrive/the-verdict.txt", "r",encoding='utf-8') as f:
  raw_text = f.read()
  print(raw_text[:1000])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?

Well!--even through th

##2. Tokenization

In [2]:
import tiktoken


In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
token_ids = tokenizer.encode(raw_text)
len(token_ids)

5145

In [9]:
tokenizer.decode(token_ids[:100])

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Th'

##3. prepare input and target data

In [13]:
# !pip install torch

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

In [39]:
class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.stride = stride
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)
    for i in range(0,len(token_ids)-max_length,stride):
      input_chunks = token_ids[i:i+max_length]
      target_chunks = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunks))
      self.target_ids.append(torch.tensor(target_chunks))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]


In [40]:
tokenizer = tiktoken.get_encoding("gpt2")


In [41]:
dataset = GPTDatasetV1(raw_text,tokenizer,10,5)
print(dataset[ 276])  #observe how getitem method is
print(len(dataset))

(tensor([ 4808, 13698, 10322,  6532,    62,  8263,    12,  9649,   550,  9258]), tensor([13698, 10322,  6532,    62,  8263,    12,  9649,   550,  9258,   284]))
1027


In [42]:
dataset.input_ids[0], dataset.target_ids[0]

(tensor([   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138]),
 tensor([  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257]))

In [43]:
dataset.input_ids[1], dataset.target_ids[1]

(tensor([ 3619,   402,   271, 10899,  2138,   257,  7026, 15632,   438,  2016]),
 tensor([  402,   271, 10899,  2138,   257,  7026, 15632,   438,  2016,   257]))

In [44]:
def create_data_loader_v1(txt,max_length,stride,batch_size,shuffle=True,drop_last=True):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt,tokenizer,max_length=max_length,stride=stride)
  data_loader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last)
  return data_loader

In [55]:
dataloader = create_data_loader_v1(raw_text,max_length=4,stride=1,batch_size=1,shuffle=False,drop_last=False)
data_iter = iter(dataloader)
next(data_iter)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [56]:
next(data_iter)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]

In [57]:
next(data_iter)

[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]

In [60]:
dataloader = create_data_loader_v1(raw_text,max_length=4,stride=4,batch_size=8,shuffle=False,drop_last=False)
data_iter = iter(dataloader)
next(data_iter)

[tensor([[   40,   367,  2885,  1464],
         [ 1807,  3619,   402,   271],
         [10899,  2138,   257,  7026],
         [15632,   438,  2016,   257],
         [  922,  5891,  1576,   438],
         [  568,   340,   373,   645],
         [ 1049,  5975,   284,   502],
         [  284,  3285,   326,    11]]),
 tensor([[  367,  2885,  1464,  1807],
         [ 3619,   402,   271, 10899],
         [ 2138,   257,  7026, 15632],
         [  438,  2016,   257,   922],
         [ 5891,  1576,   438,   568],
         [  340,   373,   645,  1049],
         [ 5975,   284,   502,   284],
         [ 3285,   326,    11,   287]])]

In [61]:
next(data_iter)

[tensor([[  287,   262,  6001,   286],
         [  465, 13476,    11,   339],
         [  550,  5710,   465, 12036],
         [   11,  6405,   257,  5527],
         [27075,    11,   290,  4920],
         [ 2241,   287,   257,  4489],
         [   64,   319,   262, 34686],
         [41976,    13,   357, 10915]]),
 tensor([[  262,  6001,   286,   465],
         [13476,    11,   339,   550],
         [ 5710,   465, 12036,    11],
         [ 6405,   257,  5527, 27075],
         [   11,   290,  4920,  2241],
         [  287,   257,  4489,    64],
         [  319,   262, 34686, 41976],
         [   13,   357, 10915,   314]])]

##4. creating token embeddings